# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [45]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aasiaat,68.7098,-52.8699,0.23,99,100,6.66,GL,1685133408
1,1,kapuskasing,49.4169,-82.4331,26.05,24,100,5.66,CA,1685133336
2,2,la grande,45.3246,-118.0877,22.23,45,89,4.03,US,1685133504
3,3,akureyri,65.6835,-18.0878,10.96,58,75,9.26,IS,1685133410
4,4,paradip garh,20.3167,86.6167,28.53,90,4,2.49,IN,1685133505


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [21]:
# %%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500    
)

# Display the map
city_data_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_weather_df.dropna(inplace=True)

# Display sample data
ideal_weather_df.head(16)

C:\Users\16133\AppData\Local\Temp\ipykernel_26896\1103565275.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_weather_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,quartu sant'elena,39.2419,9.1839,21.16,73,0,1.03,IT,1685133506
27,27,adamstown,-25.0660,-130.1015,22.59,71,0,3.84,PN,1685133275
36,36,nouadhibou,20.9310,-17.0347,21.99,78,0,2.57,MR,1685133510
101,101,vernal,40.4555,-109.5287,24.84,23,0,4.47,US,1685133524
126,126,springfield,37.2153,-93.2982,25.92,36,0,3.60,US,1685133476
209,209,descalvado,-21.9039,-47.6194,25.49,58,0,1.66,BR,1685133544
331,331,dadukou,26.5479,101.7054,21.22,61,0,0.95,CN,1685133576
353,353,prince george,53.9166,-122.7530,26.27,34,0,3.60,CA,1685133316
373,373,rawlins,41.7911,-107.2387,23.94,17,0,4.12,US,1685133590
429,429,roma,43.2128,-75.4557,21.08,33,0,4.12,US,1685133364


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head(16)

,City,Country,Lat,Lng,Humidity,Hotel Name
16,quartu sant'elena,IT,39.2419,9.1839,73,
27,adamstown,PN,-25.0660,-130.1015,71,
36,nouadhibou,MR,20.9310,-17.0347,78,
101,vernal,US,40.4555,-109.5287,23,
126,springfield,US,37.2153,-93.2982,36,
209,descalvado,BR,-21.9039,-47.6194,58,
331,dadukou,CN,26.5479,101.7054,61,
353,prince george,CA,53.9166,-122.7530,34,
373,rawlins,US,41.7911,-107.2387,17,
429,roma,US,43.2128,-75.4557,33,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [39]:
# Set parameters to search for a hotel

radius = 10000

params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
quartu sant'elena - nearest hotel: Hotel Italia
adamstown - nearest hotel: No hotel found
nouadhibou - nearest hotel: Hotel valencia
vernal - nearest hotel: Landmark Inn and Suites
springfield - nearest hotel: Hotel Vandivort
descalvado - nearest hotel: Hotel Descalvado
dadukou - nearest hotel: 锦江之星酒店
prince george - nearest hotel: Travelodge by Wyndham Prince George
rawlins - nearest hotel: Quality Inn Rawlins I-80
roma - nearest hotel: Wingate by Wyndham Rome
jamestown - nearest hotel: DoubleTree Jamestown
kani keli - nearest hotel: Hôtel Le Jardin Maoré
rushville - nearest hotel: No hotel found
port angeles - nearest hotel: Quality Inn Uptown


,City,Country,Lat,Lng,Humidity,Hotel Name
16,quartu sant'elena,IT,39.2419,9.1839,73,Hotel Italia
27,adamstown,PN,-25.0660,-130.1015,71,No hotel found
36,nouadhibou,MR,20.9310,-17.0347,78,Hotel valencia
101,vernal,US,40.4555,-109.5287,23,Landmark Inn and Suites
126,springfield,US,37.2153,-93.2982,36,Hotel Vandivort
209,descalvado,BR,-21.9039,-47.6194,58,Hotel Descalvado
331,dadukou,CN,26.5479,101.7054,61,锦江之星酒店
353,prince george,CA,53.9166,-122.7530,34,Travelodge by Wyndham Prince George
373,rawlins,US,41.7911,-107.2387,17,Quality Inn Rawlins I-80
429,roma,US,43.2128,-75.4557,33,Wingate by Wyndham Rome


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [54]:
#%%capture --no-display

# Filter the hotel_df DataFrame to include only rows with a hotel name
filtered_hotel_df = hotel_df[hotel_df['Hotel Name'].notnull()]

# Configure the map plot
filtered_hotel_map = filtered_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    hover_cols=["City", "Hotel Name", "Country"],
    scale = 1,
    color = "City",
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
)

# Display the map
filtered_hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)